# <center>Organizaci&oacute;n de Datos</center>
#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Trabajo Práctico 2 : Propiedades en Venta</center>
## <center>Parte 1</center>
### <center> Grupo 22 </center>
### <center> Integrantes: </center>
##### <center> Federico Martin Forte,  Francisco Sobral,  Ian Klaus von der Heyde,  Juan Pablo Aschieri,  Joaquin Rivero  </center>

# Carga de Datos y librerías

In [112]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import joblib
import warnings
warnings.filterwarnings('always')
#from sklearn.model_selection import train_test_split
import shapely.speedups
#from sklearn.preprocessing import StandardScaler
from IPython.display import Image
import re
from itertools import combinations
import random

In [29]:
viviendas_train=pd.read_csv('./TP1/Datasets/viviendas_caba_train.csv').copy()
viviendas_test=pd.read_csv('./TP1/Datasets/viviendas_caba_test.csv').copy()

In [30]:
viviendas = pd.concat([viviendas_train, viviendas_test]).reset_index()

# 1.Procesamiento del Lenguaje Natural

## a) Ampliación del dataset

In [31]:
veces_palabra = {}
indice_de_palabra = {}
for indice in viviendas.index:
    property_title = viviendas.loc[indice,:].property_title.lower()

    if "/" in property_title:
        property_title = property_title.replace("/"," ")
    if "." in property_title:
        property_title = property_title.replace("."," ")
    if "," in property_title:
        property_title = property_title.replace(","," ")
    if "-" in property_title:
        property_title = property_title.replace("-"," ")
    if "ó" in property_title:
        property_title = property_title.replace("ó","o")
    if "!" in property_title:
        property_title = property_title.replace("!"," ")

    property_title = property_title.split()
    
    for palabra in property_title:
        palabra = ''.join(char for char in palabra if char.isalnum())
        if palabra.isnumeric() or len(palabra) < 3:
            continue
        veces_palabra[palabra] = veces_palabra.get(palabra, 0)
        veces_palabra[palabra] +=1 

        indice_de_palabra[palabra]  = indice_de_palabra.get(palabra, list())
        indice_de_palabra[palabra].append(indice)


In [50]:
def agregar_aspecto(aspecto, palabras_clave, dataset):
    for palabra in indice_de_palabra.keys():
        for palabra_clave in palabras_clave:
            if palabra_clave in palabra:
                for i in indice_de_palabra[palabra]:
                    dataset.loc[i, aspecto] = 'sí'
    dataset[aspecto].fillna('unknown', inplace=True)

    
    print(f'cantidad de {aspecto}: { (dataset[aspecto] == "sí").sum() }')

In [33]:
palabras_a_ignorar = set(['departamento', 'depto', 'casa', 'ph', 'venta', 'amb', 'ambientes', 'con'])
for barrio in viviendas_train.barrio:
    for t in barrio.split():
        palabras_a_ignorar.add(t.lower())
for palabra in palabras_a_ignorar:
    if palabra in veces_palabra:
        veces_palabra.pop(palabra)
        indice_de_palabra.pop(palabra)

Ordenamos palabras por cantidad de apariciones para buscar posibles aspectos que sumen valor a nuestro dataset

In [53]:
sorted(veces_palabra.items(),  key=lambda x:x[1])[::-1]

[('balcon', 12291),
 ('cochera', 10563),
 ('monoambiente', 7156),
 ('terraza', 6564),
 ('estrenar', 5194),
 ('patio', 5062),
 ('amenities', 4632),
 ('luminoso', 4596),
 ('excelente', 4338),
 ('piso', 4099),
 ('frente', 3789),
 ('oportunidad', 3193),
 ('parrilla', 2833),
 ('apto', 2625),
 ('muy', 2595),
 ('vista', 2404),
 ('dependencia', 2223),
 ('baulera', 2121),
 ('semipiso', 1953),
 ('dpto', 1933),
 ('dos', 1826),
 ('reciclado', 1804),
 ('expensas', 1801),
 ('alto', 1796),
 ('hermoso', 1783),
 ('duplex', 1759),
 ('torre', 1724),
 ('profesional', 1713),
 ('dep', 1584),
 ('impecable', 1473),
 ('nuevo', 1445),
 ('coch', 1428),
 ('ambiente', 1423),
 ('hollywood', 1397),
 ('amplio', 1322),
 ('pozo', 1300),
 ('ubicacion', 1216),
 ('cocheras', 1198),
 ('propia', 1171),
 ('dormitorios', 1169),
 ('edificio', 1135),
 ('sin', 1125),
 ('pileta', 1086),
 ('gran', 1073),
 ('retasado', 977),
 ('tipo', 963),
 ('divisible', 927),
 ('chico', 923),
 ('abierta', 923),
 ('zona', 906),
 ('full', 894),
 ('

Viendo lo anterior buscamos aspectos importantes y palabras claves que los identifiquen para agregarlos al dataset:

+ A estrenar/nuevo
+ Patio/Jardin
+ balcon
+ cochera
+ pileta

In [51]:
agregar_aspecto('a_estrenar', ['estrenar', 'nuevo'], viviendas)
agregar_aspecto('patio_jardin', ['pat', 'jard'], viviendas)
agregar_aspecto('balcon', ['balc'], viviendas)
agregar_aspecto('cochera', ['coche', 'garag'], viviendas)
agregar_aspecto('pileta', ['pil', 'pisc'], viviendas)
agregar_aspecto('parrilla', ['parr'], viviendas)
agregar_aspecto('terraza', ['terra'], viviendas)
agregar_aspecto('monoambiente', ['mono'], viviendas)

cantidad de a_estrenar: 6634
cantidad de patio_jardin: 6569
cantidad de balcon: 12821
cantidad de cochera: 12375
cantidad de pileta: 1842
cantidad de parrilla: 3048
cantidad de terraza: 7517
cantidad de monoambiente: 8109


In [52]:
viviendas

,index,start_date,end_date,latitud,longitud,barrio,property_type,property_rooms,property_bedrooms,property_surface_total,...,property_title,geometry,a_estrenar,patio_jardin,balcon,cochera,pileta,parrilla,terraza,monoambiente
0,0,2021-12-15,2021-12-25,-34.555049,-58.490513,Saavedra,Departamento,2.0,1.0,58.0,...,Departamento - Saavedra,POINT (-58.4905135 -34.5550487),unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
1,1,2021-05-20,2021-06-05,-34.546991,-58.478553,Saavedra,Departamento,2.0,1.0,48.0,...,Departamento - Saavedra,POINT (-58.4785532 -34.546991),unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
2,2,2021-03-20,2021-04-16,-34.570842,-58.491805,Villa Urquiza,Departamento,2.0,1.0,62.0,...,Venta Depto 2 amb Anticipo + Cuotas CAC V. Urq...,POINT (-58.4918048 -34.5708421),unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
3,3,2021-07-17,2021-10-16,-34.589251,-58.402106,Recoleta,Departamento,3.0,2.0,60.0,...,Excelente 3 ambientes en venta en Barrio Recoleta,POINT (-58.4021058 -34.5892511),unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
4,4,2021-07-02,2022-02-09,-34.626290,-58.437632,Caballito,Departamento,3.0,2.0,67.0,...,VENTA DEPARTAMENTO 3 AMB. EN CABALLITO,POINT (-58.4376325 -34.6262899),unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92874,18571,2021-02-13,2021-03-01,-34.594510,-58.413521,Barrio Norte,Departamento,3.0,2.0,60.0,...,VENTA DEPARTAMENTO 3 AMBIENTES BALCÓN FRENTE 3...,POINT (-58.413521 -34.59451),unknown,unknown,sí,unknown,unknown,unknown,unknown,unknown
92875,18572,2021-08-20,2200-12-31,-34.540767,-58.466694,Nuñez,Casa,4.0,3.0,210.0,...,Casa - Nuñez,POINT (-58.4666936 -34.5407667),unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
92876,18573,2021-09-08,2021-10-14,-34.586723,-58.402579,Recoleta,Departamento,2.0,1.0,35.0,...,Recoleta 2 amb Oportunidad,POINT (-58.4025792 -34.5867234),unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
92877,18574,2021-01-12,2021-06-05,-34.579607,-58.489898,Villa Urquiza,Departamento,4.0,3.0,76.0,...,4 ambientes al contrafrente con balcón y coche...,POINT (-58.4898982 -34.579607),unknown,unknown,sí,sí,unknown,unknown,unknown,unknown


### PARTE B

#### UTILIZANDO LOS MISMO HIPERPARAMETROS QUE EN EL TP1

In [114]:
rand_xgb = joblib.load('./TP1/Models/rand_xgb')

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
scaler = MinMaxScaler() 
x_train_xgb = scaler.fit_transform(x_train_xgb)

In [ ]:
best_xgb_regressor = rand_xgb.best_estimator_
best_xgb_regressor

# VERIFICAR QUE ES ESTO EN LA PARTE DE XGB

In [ ]:
#Predicción con mejor estimador
y_pred_train_xgb = best_xgb_regressor.predict(viviendas)#x_train_xgb)

#Coef. de Determinación R2
r2_score_train_xgb = r2_score(y_train_xgb, y_pred_train_xgb)
print(f'Coef. de Determinación R2: {r2_score_train_xgb}')

#Coef. de Determinación R2 ajustado
n = x_train_xgb.shape[0] #n = count of rows in your dataset used for train or test
p = 4 #p = count of independent variables
print(f'Coef. de Determinación R2 ajustado: {1-(1-r2_score_train_xgb)*(n-1)/(n-p-1)}')

#Mean Square Error
mse = metrics.mean_squared_error(
        y_true  = y_train_xgb,
        y_pred  = y_pred_train_xgb,
        squared = True
       )

print(f"El error (mse) de train es: {mse}")

#Root Mean Square Error
rmse = metrics.mean_squared_error(
        y_true  = y_train_xgb,
        y_pred  = y_pred_train_xgb,
        squared = False
       )

print(f"El error (rmse) de train es: {rmse}")

## b) Modelos

# 2. Redes Neuronales

## Regresión:

## Clasificasión:

# 3. Ensamble de Modelos

## Voting

In [ ]:
import pandas as pd
import numpy as np 

import warnings
warnings.filterwarnings('ignore')


from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score


In [ ]:
#Importo los datos de entrenamiento y prueba de nuestro Random Forest

y_train = pd.read_csv('./TP1/Datasets/y_train_property_type.csv').copy()
y_test = pd.read_csv('./TP1/Datasets/y_test_property_type.csv').copy()

x_train = pd.read_csv('./TP1/Datasets/x_train_property_type.csv').copy()
x_test = pd.read_csv('./TP1/Datasets/x_test_property_type.csv').copy()


Voy a evaluar distintos clasificadores para ver cual me conviene para obtener mejores metricas.

In [ ]:
# Predicciones con decision tree classifier
dtc = DecisionTreeClassifier(random_state=42)
dtc = dtc.fit(x_train, y_train)
y_pred_dtc = dtc.predict(x_test)


# Predicciones con logistic regression model
lr = LogisticRegression()
lr = lr.fit(x_train, y_train)
y_pred_lr = lr.predict(x_test)


# Predicciones con naive bayes model
gnb = GaussianNB()
gnb = gnb.fit(x_train, y_train)
y_pred_gnb = gnb.predict(x_test)

# Predicciones con modelo AdaBoost
ada_boost_clf = AdaBoostClassifier()
ada_boost_clf.fit(x_train, y_train)
y_pred_ada = ada_boost_clf.predict(x_test)

# Predicciones con modelo Random Forest
rnd_clf = RandomForestClassifier()
rnd_clf.fit(x_train, y_train)
y_pred_rnd = rnd_clf.predict(x_test)

# Predicciones con modelo KNN
knn_clf = KNeighborsClassifier()
knn_clf.fit(x_train, y_train)
y_pred_knn = knn_clf.predict(x_test)




# evaluating the models based on their f-1 scores
f1_dtc = f1_score(y_test, y_pred_dtc,average='weighted')
f1_lr = f1_score(y_test, y_pred_lr,average='weighted')
f1_gnb = f1_score(y_test, y_pred_gnb,average='weighted')
f1_rnd = f1_score(y_test, y_pred_rnd,average='weighted')
f1_knn = f1_score(y_test, y_pred_knn,average='weighted')
f1_ada = f1_score(y_test, y_pred_ada,average='weighted')

# print the f-1 scores
print('F1-score of decision tree classifier: {}'.format(np.round(f1_dtc,4)))
print('F1-score of support vector machine: {}'.format(np.round(f1_lr,4)))
print('F1-score of naive bayes classifier: {}'.format(np.round(f1_gnb,4)))
print('F1-score of random forest classifier: {}'.format(np.round(f1_rnd,4)))
print('F1-score of KNN classifier: {}'.format(np.round(f1_knn,4)))
print('F1-score of AdaBoost classifier: {}'.format(np.round(f1_ada,4)))

F1-score of decision tree classifier: 0.7051
F1-score of support vector machine: 0.5457
F1-score of naive bayes classifier: 0.3733
F1-score of random forest classifier: 0.7451
F1-score of KNN classifier: 0.6111
F1-score of AdaBoost classifier: 0.5932


Aca podemos ver que los mejores 3 modelos fueron los de Decision Tree Classifier, Random Forest Classifier y KNN.
Procedo a realizar el Voting con dichos tres modelos.
Para obtener mejores resultados, tambien voy a evaluar realizando una votacion de tipo Hard y una de tipo Soft y utilizar la que mejores resultados de.

In [ ]:
#Creo clasificadores

dtc_clf = DecisionTreeClassifier()
rnd_clf = RandomForestClassifier()
knn_clf = KNeighborsClassifier()

#Creo ensemble de Votación Hard
vot_clf_h = VotingClassifier(estimators = [('dtc', dtc_clf), ('rnd', rnd_clf), ('knn', knn_clf)], voting = 'hard')

#Creo ensemble de Votación Soft
vot_clf_s = VotingClassifier(estimators = [('dtc', dtc_clf), ('rnd', rnd_clf), ('knn', knn_clf)], voting = 'soft')


#Entreno ambos ensembles
vot_clf_h.fit(x_train, y_train)
vot_clf_s.fit(x_train, y_train)

#Evaluo en conjunto de test
pred_h = vot_clf_h.predict(x_test)
pred_s = vot_clf_s.predict(x_test)



#Calculo accuracy
print("RESULTADOS HARD VOTING:\n")
print('Accuracy Score: {}'.format(accuracy_score(y_test, pred_h)))
print('F1-score: {}'.format(f1_score(y_test, pred_h,average="weighted")))
print("Precision Score : ",precision_score(y_test,pred_h,pos_label='positive',average="weighted"))
print("Recall Score :" , recall_score(y_test, pred_h, pos_label='positive',average="weighted"))

print("\nRESULTADOS SOFT VOTING:\n")
print('Accuracy Score: {}'.format(accuracy_score(y_test, pred_s)))
print('F1-score: {}'.format(f1_score(y_test, pred_s,average="weighted")))
print("Precision Score: ",precision_score(y_test,pred_s,pos_label='positive',average="weighted"))
print("Recall Score:" , recall_score(y_test, pred_s, pos_label='positive',average="weighted"))

RESULTADOS HARD VOTING:

Accuracy Score: 0.7330426356589147
F1-score: 0.7325673315978177
Precision Score :  0.7336933775486353
Recall Score : 0.7330426356589147

RESULTADOS SOFT VOTING:

Accuracy Score: 0.7258828596037898
F1-score: 0.7253054999623219
Precision Score :  0.7260450948692784
Recall Score : 0.7258828596037898


Concluimos que conviene utilizar el metodo de Hard voting.

## Stacking

In [ ]:
import numpy as np 
import pandas as pd
from collections import defaultdict
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegressionCV

warnings.filterwarnings('ignore')


from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [ ]:
#Importo los datos de entrenamiento y prueba de nuestro Random Forest

y_train = pd.read_csv('./TP1/Datasets/y_train_property_type.csv').copy()
y_test = pd.read_csv('./TP1/Datasets/y_test_property_type.csv').copy()

x_train = pd.read_csv('./TP1/Datasets/x_train_property_type.csv').copy()
x_test = pd.read_csv('./TP1/Datasets/x_test_property_type.csv').copy()


In [ ]:
#Funcion para evaluar los modelos 5fold -CV (2 repeticiones)
def evaluate_model(model, x, y):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
    scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, verbose=1, n_jobs=3, error_score='raise')
    return scores

In [ ]:
#Funcion para plotear a performance
def plot_results(model_scores, name):
    
    model_names = list(model_scores.keys())
    results = [model_scores[model] for model in model_names]
    fig = go.Figure()
    for model, result in zip(model_names, results):
        fig.add_trace(go.Box(
            y=result,
            name=model,
            boxpoints='all',
            jitter=0.5,
            whiskerwidth=0.2,
            marker_size=2,
            line_width=1)
        )
    
    fig.update_layout(
    title='Performance of Different Models Using 5-Fold Cross-Validation',
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    xaxis_title='Model',
    yaxis_title='Accuracy',
    showlegend=False)
    fig.show()

In [ ]:
#Modelos Base
base_models = {'random_forest':RandomForestClassifier(n_estimators=50),
               'dtc': DecisionTreeClassifier(),
               'knn': KNeighborsClassifier(n_neighbors=11)
               } 

#Guarda los scores de cada modelo
model_scores = defaultdict()

#Entreno los clasificadores
for name, model in base_models.items():
    print('Evaluating {}'.format(name))
    scores = evaluate_model(model, x_train, y_train)
    model_scores[name] = scores

Evaluating random_forest


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:   37.5s finished


Evaluating dtc


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    3.4s finished


Evaluating knn


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:  2.8min finished


In [ ]:
from sklearn.ensemble import StackingClassifier

#Modelos Base
base_models = [('random_forest', RandomForestClassifier(n_estimators=50)),
               ('dtc', DecisionTreeClassifier()),
               ('knn', KNeighborsClassifier(n_neighbors=11))]

#Meta Modelo
meta_model = LogisticRegressionCV()

#Ensemble Stacking
stacking_model = StackingClassifier(estimators=base_models, 
                                    final_estimator=meta_model, 
                                    passthrough=True,
                                    cv=5,
                                    verbose=2)

In [ ]:
#Performance el modelo en entrenamiento
stacking_scores = evaluate_model(stacking_model, x_train, y_train)
model_scores['stacking'] = stacking_scores

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed: 25.7min finished


In [ ]:
#Grafico performance
plot_results(model_scores, name='stacking_model_cv')

Tras multiples pruebas de modelos, concluimos que estos tres (Random Forest, Decision Tree y KNN) son los mejores modelos para elegir. Fueron los que mejores métricas nos dieron.

In [ ]:
#Evaluo en conjunto de test
#stacking_model.fit(x_train,y_train).score(x_test,y_test)
stacking_model.fit(x_train,y_train)
y_pred_st = stacking_model.predict(x_test)


print('Accuracy Score: {}'.format(accuracy_score(y_test, y_pred_st)))
print('F1-score: {}'.format(f1_score(y_test, y_pred_st,average="weighted")))
print("Precision Score: ",precision_score(y_test,y_pred_st,pos_label='positive',average="weighted"))
print("Recall Score:" , recall_score(y_test, y_pred_st, pos_label='positive',average="weighted"))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   25.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.3min finished


Accuracy Score: 0.7502153316106804
F1-score: 0.7490939091565331
Precision Score:  0.7535181655904228
Recall Score: 0.7502153316106804


In [ ]:
import joblib
joblib.dump(base_models, './Models/base_models')
joblib.dump(model_scores, './Models/model_scores')
joblib.dump(meta_model, './Models/meta_model')
joblib.dump(stacking_model, './Models/stacking_model')
joblib.dump(stacking_scores, './Models/stacking_scores')
joblib.dump(y_pred_st, './Models/y_pred_st')

['./Models/y_pred_st']

A modo de conclusion, podemos decir que, como era de esperarse, las metricas de Stacking fueron superiores a las de Voting.